In [1]:
##Get the top two product name in each category based on the maximum order item subtotal


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()


In [3]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window as w

In [4]:
o_items= spark.read.format('csv').options(header='True',inferSchema='true').load('/FileStore/tables/order_items.csv')
o_items=o_items.select('_c0')
o_items.show(5,False)

In [5]:
o=o_items.select(
f.split(o_items._c0,',')[0].cast('int').alias('o_item_id'),
f.split(o_items._c0,',')[1].cast('int').alias('o_item_order_id'),
f.split(o_items._c0,',')[2].cast('int').alias('o_item_product_id'),
f.split(o_items._c0,',')[3].cast('int').alias('quantity'),
f.split(o_items._c0,',')[4].cast('float').alias('subtotal'),
f.split(o_items._c0,',')[5].cast('float').alias('product_price')
  )

In [6]:
o.show(5,False)

In [7]:
products= spark.read.format('csv').options(header='False',inferSchema='true').load('/FileStore/tables/products.csv')


In [8]:
p=products.select(
f.split(products._c0,',')[0].cast('int').alias('product_id'),
f.split(products._c0,',')[1].cast('int').alias('product_cat_id'),
f.split(products._c0,',')[2].cast('string').alias('product_name'),
f.split(products._c0,',')[3].cast('string').alias('description'),
f.split(products._c0,',')[4].cast('float').alias('price'),
f.split(products._c0,',')[5].cast('string').alias('image')
)

In [9]:
p.show(3,False)

In [10]:
categories= spark.read.format('csv').options(header='False',inferSchema='true').load('/FileStore/tables/categories.csv')



In [11]:
c=categories.select(
f.split(categories._c0,',')[0].cast('int').alias('category_id'),
f.split(categories._c0,',')[1].cast('int').alias('category_dept_id'),
f.split(categories._c0,',')[2].cast('string').alias('category_name')
)

In [12]:
c.show(4,False)

In [13]:
#dense_rank
o.join(p,p.product_id==o.o_item_product_id,"inner").join(c,c.category_id==p.product_cat_id,"inner").\
groupBy(c.category_name,p.product_name).\
agg(f.max(o.subtotal).alias("max_subtotal")).\
sort(f.asc("category_name"),f.desc("max_subtotal")).\
select("category_name","product_name","max_subtotal",f.dense_rank().over(w.partitionBy("category_name").orderBy(f.desc("max_subtotal"))).alias("rank")).\
filter("rank<=2").select("category_name","product_name","max_subtotal").show(truncate=False)

In [14]:
#rank
o.join(p,p.product_id==o.o_item_product_id,"inner").join(c,c.category_id==p.product_cat_id,"inner").\
groupBy(c.category_name,p.product_name).\
agg(f.max(o.subtotal).alias("max_subtotal")).\
sort(f.asc("category_name"),f.desc("max_subtotal")).\
select("category_name","product_name","max_subtotal",f.rank().over(w.partitionBy("category_name").orderBy(f.desc("max_subtotal"))).alias("rank")).\
filter("rank<=2").select("category_name","product_name","max_subtotal").show(truncate=False)